In [1]:
!pip install pymorphy2 > None
!pip install pymystem3

In [2]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
import nltk
from string import punctuation
import pymorphy2
from nltk.stem.snowball import RussianStemmer
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt')
from pymystem3 import Mystem
from sklearn.mixture import GaussianMixture

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
def hh_job_preparation_was(string):
    global global_counter_modify_jobs

    result = string.lower()

    temp_string = result

    result = result.replace("зам. ", "заместитель ")
    result = result.replace("нач. ", "начальник ")
    result = result.replace("рук. ", "руководитель ")
    result = result.replace("ген. ", "генеральный ")
    result = result.replace("гендиректор", "генеральный директор")
    result=result.replace("вет.", "ветеринарный")
    result = result.replace("ветврач", "ветеринарный врач")
    result = result.replace("гл.", "главный")
    result = result.replace("гл", "главный")
    result = result.replace("–", "-")
    result = result.replace(" - ", "-")

    if (result.find("(")>0 and result.find(")")>0):
        temp = ""
        not_closed = False
        for i in range(len(result)):
            if result[i] == "(":
                not_closed = True
            elif result[i] == ")":
                not_closed = False
            else:
                if not not_closed:
                    temp += result[i]
        result = temp

    if (result.find("«")>0 and result.find("»")>0):
        temp = ""
        not_closed = False
        for i in range(len(result)):
            if result[i] == "«":
                not_closed = True
            elif result[i] == "»":
                not_closed = False
            else:
                if not not_closed:
                    temp += result[i]
        result = temp


    if (result.find(",") > 0) :
        result=result.partition(',')[0]

    if (result.find(";") > 0) :
        result=result.partition(';')[0]

    if (result.find(".") > 0) :
        result=result.partition('.')[0]

    if (result.find("/") > 0) :
        result=result.partition('/')[0]

    if (result.find(" по ") > 0) :
        result=result.partition(' по ')[0]


    if (result.find("начальник ") > -1 or result.find("руководитель ") > -1):
        if (result.find("отдела") > -1):
            result = "начальник отдела"
        elif (result.find("службы") > -1):
            result = "начальник службы"
        elif (result.find("управления") > -1):
            result = "начальник управления"
        elif (result.find("подразделения") > -1):
            result = "начальник подразделения"
        elif (result.find("группы") > -1):
            result = "начальник группы"
    if (result.find("агроном") > -1):
      result = "начальник отдела"
    if (result.find("ветеринарный врач") > -1):
      result = "ветеринарный врач"
    if (result.find("бригадир") > -1):
      result = "бригадир"
    for i in range(10):
        if str(i) in result:
          result=result[:result.find(str(i))]

    if (result.find("заместитель начальника") > -1 or result.find("заместитель руководителя ") > -1):
        if (result.find("отдела") > -1):
            result = "заместитель начальника отдела"
        elif (result.find("службы") > -1):
            result = "заместитель начальника службы"
        elif (result.find("управления") > -1):
            result = "заместитель начальника управления"
        elif (result.find("подразделения") > -1):
            result = "заместитель начальника подразделения"
        elif (result.find("группы") > -1):
            result = "заместитель начальника группы"

    if (result.find("developer") > -1 or result.find("programmer") > -1 ):
        result="программист"

    if (result.find("генеральный директор") > -1 or result.find("programmer") > -1 ):
        result="генеральный директор"

    result=result.strip()

    if (result != temp_string):
        global_counter_modify_jobs = global_counter_modify_jobs +1


    return(result)

In [4]:
punct= list(punctuation)
punct.append(' ')
stopwords = nltk.corpus.stopwords.words('russian')
stopwords.append('\n')
def hh_description_prepar(string):
    result = string.lower()
    txt= word_tokenize(result)          #токенизация
    txt = [i for i in txt if i not in punct and i not in stopwords]
    #lemm=Mystem()
    #txt = lemm.lemmatize(" ".join(txt))      #лемматизация
    morph = pymorphy2.MorphAnalyzer()
    lemms_text = [morph.parse(i)[0].normal_form for i in txt]
    stemmer = RussianStemmer()
    stems_lemms_low_txt_punct_stopword = [stemmer.stem(i) for i in lemms_text]    #стемминг
    return " ".join(txt)

In [5]:
data_2=pd.read_json('data_frame2.json')

In [ ]:
global_counter_modify_jobs=0
corpus_prof_1=data_2['Prof_1']
corpus_prof_2=data_2['Prof_2']

corpus_new_prof_1=[ hh_job_preparation_was(i) for  i in corpus_prof_1]
corpus_new_prof_2=[ hh_job_preparation_was(i) for  i in corpus_prof_2]
corpus_desc=data_2['Description_1']
corpus_new_desc=[ hh_description_prepar(i) for  i in corpus_desc ]


In [7]:
vectorizer = TfidfVectorizer()
TF_IDF_vector_prof_1= vectorizer.fit_transform(corpus_prof_1).toarray()

TF_IDF_vector_new_prof_1=[]
for i in TF_IDF_vector_prof_1:            #создаю копию списка в более удобном формате
  part=[]
  for j in i:
    part.append(j)
  TF_IDF_vector_new_prof_1.append(part)
TF_IDF_vector_prof_2= vectorizer.fit_transform(corpus_prof_2).toarray()
TF_IDF_vector_new_prof_2=[]
for i in TF_IDF_vector_prof_2:            #создаю копию списка в более удобном формате
  part=[]
  for j in i:
    part.append(j)
  TF_IDF_vector_new_prof_2.append(part)

In [8]:
vectorizer = TfidfVectorizer()

TF_IDF_vector_desc= vectorizer.fit_transform(corpus_new_desc).toarray()
TF_IDF_vector_new_desc=[]
for i in TF_IDF_vector_desc:            #создаю копию списка в более удобном формате
  part=[]
  for j in i:
    part.append(j)
  TF_IDF_vector_new_desc.append(part)


In [9]:
data_2['Prepare prof_1']=corpus_prof_1
data_2['Prepare prof_2']=corpus_prof_2
data_2['Prepare prof_1 tfidf']=TF_IDF_vector_new_prof_1
data_2['Prepare prof_2 tfidf']=TF_IDF_vector_new_prof_2
data_2['Prepare description']=corpus_new_desc
data_2['Prepare description tfidf']=TF_IDF_vector_new_desc

data_2.to_json('/content/data_with_prepare_job.json')